In [1]:
import math
import os
import time
import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

In [3]:
def load_data(path):
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0'])
    data = data[1:]
    print(data.shape, (data == 0).sum().sum())
    return data

In [4]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
    return trainX, trainY, testX, testY

In [5]:
def generate_model(seq_len, node_num, pre_len):
    model = tf.keras.Sequential([
        tf.keras.layers.GRU(units=64, return_sequences=True, input_shape = [seq_len, node_num]),
        tf.keras.layers.GRU(units=64),
        tf.keras.layers.Dense(pre_len * node_num)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [6]:
def train_model(model, train_datas, realY):
    trainX, trainY, testX, testY = train_datas[0], train_datas[1], train_datas[2], train_datas[3]
    print(trainX.shape, trainY.shape, testX.shape, testY.shape, realY.shape)
    
    model.fit(trainX, trainY, epochs=20, batch_size=32)
    y_pred = model.predict(testX)
    print(y_pred.shape)
    
    rmse = math.sqrt(mean_squared_error(realY, y_pred))
    mae = mean_absolute_error(realY, y_pred)
    mape = mean_absolute_percentage_error(realY, y_pred)
    f_norm = la.norm(realY - y_pred) / la.norm(realY)
    acc = 1-f_norm
    
    print(rmse, mae, mape, acc)

In [7]:
path_0 = '../Data/PEMS-BAY/speed_bay_0.csv'
path_5 = '../Data/PEMS-BAY/speed_bay_5.csv'
path_10 = '../Data/PEMS-BAY/speed_bay_10.csv'
path_20 = '../Data/PEMS-BAY/speed_bay_20.csv'

In [8]:
train_rate = 0.8
seq_len = 12

In [9]:
mainData = load_data(path_0)
data_0 = load_data(path_0)
data_5 = load_data(path_5)
data_10 = load_data(path_10)
data_20 = load_data(path_20)

(52115, 325) 0
(52115, 325) 0
(52115, 325) 833840
(52115, 325) 1667680
(52115, 325) 3387475


# Pred = 15min

In [10]:
pre_len = 3

In [11]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

52115 325


(10408, 975)

### Unobserved nodes = 0%

In [12]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975)
(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975) (10408, 975)
Epoch 1/20
1303/1303 [==============================] - 17s 10ms/step - loss: 1213.2606 - mae: 30.5899
Epoch 2/20
1303/1303 [==============================] - 11s 8ms/step - loss: 93.3513 - mae: 6.5989
Epoch 3/20
1303/1303 [==============================] - 12s 9ms/step - loss: 84.9942 - mae: 5.3785
Epoch 4/20
1303/1303 [==============================] - 10s 8ms/step - loss: 84.9994 - mae: 5.3710
Epoch 5/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0062 - mae: 5.3752
Epoch 6/20
1303/1303 [==============================] - 11s 8ms/step - loss: 85.0264 - mae: 5.3723
Epoch 7/20
1303/1303 [==============================] - 13s 10ms/step - loss: 85.0352 - mae: 5.3698
Epoch 8/20
1303/1303 [==============================] - 11s 8ms/step - loss: 85.0317 - mae: 5.3762
Epoch 9/20
1303/1303 [==============================] - 13s 10ms/step -

### Unobserved nodes = 5%

In [13]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975)
(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975) (10408, 975)
Epoch 1/20
1303/1303 [==============================] - 14s 9ms/step - loss: 1147.8242 - mae: 28.9897
Epoch 2/20
1303/1303 [==============================] - 13s 10ms/step - loss: 88.6340 - mae: 6.2670
Epoch 3/20
1303/1303 [==============================] - 12s 9ms/step - loss: 80.8739 - mae: 5.1250
Epoch 4/20
1303/1303 [==============================] - 12s 9ms/step - loss: 80.8793 - mae: 5.1203
Epoch 5/20
1303/1303 [==============================] - 12s 10ms/step - loss: 80.8874 - mae: 5.1135
Epoch 6/20
1303/1303 [==============================] - 7s 5ms/step - loss: 80.8908 - mae: 5.1207
Epoch 7/20
1303/1303 [==============================] - 10s 8ms/step - loss: 80.9092 - mae: 5.1177
Epoch 8/20
1303/1303 [==============================] - 11s 8ms/step - loss: 80.9149 - mae: 5.1171
Epoch 9/20
1303/1303 [==============================] - 13s 10ms/step -

### Unobserved nodes = 10%

In [14]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975)
(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975) (10408, 975)
Epoch 1/20
1303/1303 [==============================] - 12s 7ms/step - loss: 1089.5057 - mae: 27.5028
Epoch 2/20
1303/1303 [==============================] - 9s 7ms/step - loss: 83.6044 - mae: 5.9246
Epoch 3/20
1303/1303 [==============================] - 10s 7ms/step - loss: 76.2578 - mae: 4.8230
Epoch 4/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2625 - mae: 4.8271
Epoch 5/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2710 - mae: 4.8252
Epoch 6/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2864 - mae: 4.8254
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2878 - mae: 4.8266
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2917 - mae: 4.8264
Epoch 9/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76

### Unobserved nodes = 20%

In [15]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975)
(41677, 12, 325) (41677, 975) (10408, 12, 325) (10408, 975) (10408, 975)
Epoch 1/20
1303/1303 [==============================] - 12s 7ms/step - loss: 966.7930 - mae: 24.4110
Epoch 2/20
1303/1303 [==============================] - 9s 7ms/step - loss: 75.8443 - mae: 5.3410
Epoch 3/20
1303/1303 [==============================] - 9s 7ms/step - loss: 69.2816 - mae: 4.3654
Epoch 4/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2850 - mae: 4.3676
Epoch 5/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2969 - mae: 4.3647
Epoch 6/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2981 - mae: 4.3703
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 69.3051 - mae: 4.3656
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 69.3188 - mae: 4.3688
Epoch 9/20
1303/1303 [==============================] - 9s 7ms/step - loss: 69.3

# Pred = 30min

In [16]:
pre_len = 6

In [17]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

52115 325


(10405, 1950)

### Unobserved nodes = 0%

In [18]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950)
(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950) (10405, 1950)
Epoch 1/20
1303/1303 [==============================] - 12s 7ms/step - loss: 1216.1260 - mae: 30.6360
Epoch 2/20
1303/1303 [==============================] - 9s 7ms/step - loss: 93.4049 - mae: 6.6104
Epoch 3/20
1303/1303 [==============================] - 10s 8ms/step - loss: 84.9924 - mae: 5.3764
Epoch 4/20
1303/1303 [==============================] - 9s 7ms/step - loss: 84.9967 - mae: 5.3698
Epoch 5/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0131 - mae: 5.3716
Epoch 6/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0222 - mae: 5.3743
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0281 - mae: 5.3732
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0280 - mae: 5.3751
Epoch 9/20
1303/1303 [==============================] - 9s 7ms/step - los

### Unobserved nodes = 5%

In [19]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950)
(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950) (10405, 1950)
Epoch 1/20
1303/1303 [==============================] - 12s 7ms/step - loss: 1157.6827 - mae: 29.1464
Epoch 2/20
1303/1303 [==============================] - 9s 7ms/step - loss: 88.9413 - mae: 6.3025
Epoch 3/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.8687 - mae: 5.1235
Epoch 4/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.8798 - mae: 5.1164
Epoch 5/20
1303/1303 [==============================] - 5s 4ms/step - loss: 80.8877 - mae: 5.1180
Epoch 6/20
1303/1303 [==============================] - 6s 4ms/step - loss: 80.8895 - mae: 5.1192
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.8992 - mae: 5.1178
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.9120 - mae: 5.1189
Epoch 9/20
1303/1303 [==============================] - 10s 7ms/step - los

### Unobserved nodes = 10%

In [20]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950)
(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950) (10405, 1950)
Epoch 1/20
1303/1303 [==============================] - 12s 7ms/step - loss: 1093.4764 - mae: 27.5724
Epoch 2/20
1303/1303 [==============================] - 10s 7ms/step - loss: 83.7650 - mae: 5.9365
Epoch 3/20
1303/1303 [==============================] - 10s 8ms/step - loss: 76.2526 - mae: 4.8253
Epoch 4/20
1303/1303 [==============================] - 10s 7ms/step - loss: 76.2613 - mae: 4.8271
Epoch 5/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2733 - mae: 4.8235
Epoch 6/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2734 - mae: 4.8283
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2936 - mae: 4.8236
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2886 - mae: 4.8260
Epoch 9/20
1303/1303 [==============================] - 9s 7ms/step - l

### Unobserved nodes = 20%

In [21]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950)
(41674, 12, 325) (41674, 1950) (10405, 12, 325) (10405, 1950) (10405, 1950)
Epoch 1/20
1303/1303 [==============================] - 12s 7ms/step - loss: 961.8629 - mae: 24.3267
Epoch 2/20
1303/1303 [==============================] - 9s 7ms/step - loss: 75.6019 - mae: 5.3207
Epoch 3/20
1303/1303 [==============================] - 8s 6ms/step - loss: 69.2824 - mae: 4.3674
Epoch 4/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2842 - mae: 4.3673
Epoch 5/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2961 - mae: 4.3641
Epoch 6/20
1303/1303 [==============================] - 6s 4ms/step - loss: 69.3014 - mae: 4.3705
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 69.3068 - mae: 4.3657
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 69.3124 - mae: 4.3676
Epoch 9/20
1303/1303 [==============================] - 9s 7ms/step - loss:

# Pred = 45min

In [22]:
pre_len = 9

In [23]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

52115 325


(10402, 2925)

### Unobserved nodes = 0%

In [24]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925)
(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925) (10402, 2925)
Epoch 1/20
1303/1303 [==============================] - 11s 7ms/step - loss: 1212.7516 - mae: 30.5764
Epoch 2/20
1303/1303 [==============================] - 9s 7ms/step - loss: 93.2401 - mae: 6.5931
Epoch 3/20
1303/1303 [==============================] - 9s 7ms/step - loss: 84.9858 - mae: 5.3756
Epoch 4/20
1303/1303 [==============================] - 9s 7ms/step - loss: 84.9927 - mae: 5.3722
Epoch 5/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0054 - mae: 5.3709
Epoch 6/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0029 - mae: 5.3743
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0269 - mae: 5.3737
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 85.0296 - mae: 5.3732
Epoch 9/20
1303/1303 [==============================] - 9s 7ms/step - loss

### Unobserved nodes = 5%

In [25]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925)
(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925) (10402, 2925)
Epoch 1/20
1303/1303 [==============================] - 8s 4ms/step - loss: 1156.1705 - mae: 29.1328
Epoch 2/20
1303/1303 [==============================] - 5s 4ms/step - loss: 89.0081 - mae: 6.3040
Epoch 3/20
1303/1303 [==============================] - 7s 6ms/step - loss: 80.8648 - mae: 5.1222
Epoch 4/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.8720 - mae: 5.1137
Epoch 5/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.8809 - mae: 5.1213
Epoch 6/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.8852 - mae: 5.1147
Epoch 7/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.8792 - mae: 5.1200
Epoch 8/20
1303/1303 [==============================] - 9s 7ms/step - loss: 80.9026 - mae: 5.1164
Epoch 9/20
1303/1303 [==============================] - 9s 7ms/step - loss:

### Unobserved nodes = 10%

In [26]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925)
(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925) (10402, 2925)
Epoch 1/20
1303/1303 [==============================] - 11s 7ms/step - loss: 1095.9984 - mae: 27.6113
Epoch 2/20
1303/1303 [==============================] - 9s 7ms/step - loss: 83.7545 - mae: 5.9386
Epoch 3/20
1303/1303 [==============================] - 9s 7ms/step - loss: 76.2530 - mae: 4.8269
Epoch 4/20
1303/1303 [==============================] - 8s 6ms/step - loss: 76.2611 - mae: 4.8240
Epoch 5/20
1303/1303 [==============================] - 5s 4ms/step - loss: 76.2662 - mae: 4.8253
Epoch 6/20
1303/1303 [==============================] - 5s 4ms/step - loss: 76.2760 - mae: 4.8264
Epoch 7/20
1303/1303 [==============================] - 5s 4ms/step - loss: 76.2805 - mae: 4.8274
Epoch 8/20
1303/1303 [==============================] - 5s 4ms/step - loss: 76.2869 - mae: 4.8264
Epoch 9/20
1303/1303 [==============================] - 5s 4ms/step - loss

### Unobserved nodes = 20%

In [27]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925)
(41671, 12, 325) (41671, 2925) (10402, 12, 325) (10402, 2925) (10402, 2925)
Epoch 1/20
1303/1303 [==============================] - 7s 4ms/step - loss: 965.2993 - mae: 24.3906
Epoch 2/20
1303/1303 [==============================] - 5s 4ms/step - loss: 75.7373 - mae: 5.3326
Epoch 3/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2749 - mae: 4.3660
Epoch 4/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2843 - mae: 4.3688
Epoch 5/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2880 - mae: 4.3650
Epoch 6/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2958 - mae: 4.3664
Epoch 7/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.3016 - mae: 4.3666
Epoch 8/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.3031 - mae: 4.3695
Epoch 9/20
1303/1303 [==============================] - 5s 4ms/step - loss: 

# Pred = 60min

In [28]:
pre_len = 12

In [29]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

52115 325


(10399, 3900)

### Unobserved nodes = 0%

In [30]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900)
(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900) (10399, 3900)
Epoch 1/20
1303/1303 [==============================] - 7s 4ms/step - loss: 1221.2584 - mae: 30.7208
Epoch 2/20
1303/1303 [==============================] - 5s 4ms/step - loss: 93.5188 - mae: 6.6197
Epoch 3/20
1303/1303 [==============================] - 5s 4ms/step - loss: 84.9754 - mae: 5.3768
Epoch 4/20
1303/1303 [==============================] - 5s 4ms/step - loss: 84.9881 - mae: 5.3765
Epoch 5/20
1303/1303 [==============================] - 5s 4ms/step - loss: 84.9926 - mae: 5.3714
Epoch 6/20
1303/1303 [==============================] - 5s 4ms/step - loss: 85.0039 - mae: 5.3679
Epoch 7/20
1303/1303 [==============================] - 5s 4ms/step - loss: 85.0068 - mae: 5.3751
Epoch 8/20
1303/1303 [==============================] - 5s 4ms/step - loss: 85.0183 - mae: 5.3757
Epoch 9/20
1303/1303 [==============================] - 5s 4ms/step - loss:

### Unobserved nodes = 5%

In [31]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900)
(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900) (10399, 3900)
Epoch 1/20
1303/1303 [==============================] - 7s 4ms/step - loss: 1153.6931 - mae: 29.0805
Epoch 2/20
1303/1303 [==============================] - 5s 4ms/step - loss: 88.6844 - mae: 6.2863
Epoch 3/20
1303/1303 [==============================] - 5s 4ms/step - loss: 80.8542 - mae: 5.1109
Epoch 4/20
1303/1303 [==============================] - 5s 4ms/step - loss: 80.8616 - mae: 5.1197
Epoch 5/20
1303/1303 [==============================] - 4s 3ms/step - loss: 80.8794 - mae: 5.1189
Epoch 6/20
1303/1303 [==============================] - 4s 3ms/step - loss: 80.8863 - mae: 5.1174
Epoch 7/20
1303/1303 [==============================] - 4s 3ms/step - loss: 80.8926 - mae: 5.1147
Epoch 8/20
1303/1303 [==============================] - 4s 3ms/step - loss: 80.8945 - mae: 5.1211
Epoch 9/20
1303/1303 [==============================] - 5s 3ms/step - loss:

### Unobserved nodes = 10%

In [32]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900)
(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900) (10399, 3900)
Epoch 1/20
1303/1303 [==============================] - 7s 4ms/step - loss: 1098.4819 - mae: 27.6547
Epoch 2/20
1303/1303 [==============================] - 4s 3ms/step - loss: 83.8393 - mae: 5.9454
Epoch 3/20
1303/1303 [==============================] - 4s 3ms/step - loss: 76.2414 - mae: 4.8259
Epoch 4/20
1303/1303 [==============================] - 4s 3ms/step - loss: 76.2455 - mae: 4.8262
Epoch 5/20
1303/1303 [==============================] - 4s 3ms/step - loss: 76.2617 - mae: 4.8234
Epoch 6/20
1303/1303 [==============================] - 5s 4ms/step - loss: 76.2725 - mae: 4.8240
Epoch 7/20
1303/1303 [==============================] - 5s 4ms/step - loss: 76.2697 - mae: 4.8272
Epoch 8/20
1303/1303 [==============================] - 4s 3ms/step - loss: 76.2777 - mae: 4.8249
Epoch 9/20
1303/1303 [==============================] - 4s 3ms/step - loss:

### Unobserved nodes = 20%

In [33]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900)
(41668, 12, 325) (41668, 3900) (10399, 12, 325) (10399, 3900) (10399, 3900)
Epoch 1/20
1303/1303 [==============================] - 7s 4ms/step - loss: 965.1754 - mae: 24.3855
Epoch 2/20
1303/1303 [==============================] - 5s 4ms/step - loss: 75.7137 - mae: 5.3265
Epoch 3/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2687 - mae: 4.3689
Epoch 4/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2734 - mae: 4.3672
Epoch 5/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2803 - mae: 4.3676
Epoch 6/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2925 - mae: 4.3631
Epoch 7/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.2926 - mae: 4.3651
Epoch 8/20
1303/1303 [==============================] - 5s 4ms/step - loss: 69.3033 - mae: 4.3707
Epoch 9/20
1303/1303 [==============================] - 5s 4ms/step - loss: 